# <center> <b> <h1>   SVM RFE Non Linear (All Features) </h1></b></center>

In [1]:
from sklearn.metrics import accuracy_score, r2_score
import joblib
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

import pandas as pd

PATH_DATASETS = "../../2_Data_preparation/2_Split_Raw_Datasets/output/"

In [2]:
def readCsvs(split):
    datasets = []
    constant1 = 'X_'
    constant2 = '_' + split
    features = ['area', 'meancurv', 'thickness', 'volume']

    for feature in features:
        df = pd.read_csv(PATH_DATASETS + constant1 + feature + constant2 +'.csv', index_col=0)
        datasets.append({'name': feature, 'data': df})
    return datasets

# 1. Prepare the Data

In [3]:
df_X_tr = readCsvs('train')
df_X_tr[0]['data'].shape

(550, 148)

In [4]:
df_y_train = pd.read_csv(PATH_DATASETS + 'Y_train.csv', index_col = 0)
df_y_train.shape

(550, 1)

In [5]:
df_X_val = readCsvs('validate')
df_X_val[0]['data'].shape

(338, 148)

In [6]:
df_y_val = pd.read_csv(PATH_DATASETS + 'Y_validate.csv', index_col = 0)
df_y_val.shape

(338, 1)

# Normalize  data

In [7]:
for df_tr, df_val in zip(df_X_tr, df_X_val):
    train = df_tr['data']
    val = df_val['data']
    
    normal_tr = (train - train.mean(axis=0))/train.std()
    normal_val = (val - val.mean(axis=0))/val.std()
    
    df_tr['data'] = normal_tr
    df_val['data'] = normal_val

In [8]:
df_X_tr[0]['data'].head()

,lh_G_and_S_frontomargin_area,lh_G_and_S_occipital_inf_area,lh_G_and_S_paracentral_area,lh_G_and_S_subcentral_area,lh_G_and_S_transv_frontopol_area,lh_G_and_S_cingul-Ant_area,lh_G_and_S_cingul-Mid-Ant_area,lh_G_and_S_cingul-Mid-Post_area,lh_G_cingul-Post-dorsal_area,lh_G_cingul-Post-ventral_area,...,rh_S_parieto_occipital_area,rh_S_pericallosal_area,rh_S_postcentral_area,rh_S_precentral-inf-part_area,rh_S_precentral-sup-part_area,rh_S_suborbital_area,rh_S_subparietal_area,rh_S_temporal_inf_area,rh_S_temporal_sup_area,rh_S_temporal_transverse_area
414229,0.309576,0.224009,1.107822,1.264846,-0.053643,0.447702,0.977805,0.781809,-0.594180,-0.761092,...,0.221416,0.381852,-0.114597,1.444931,0.064915,-0.435619,-1.510187,-0.718114,-1.077114,-0.575371
541943,-0.076249,0.352958,-0.262869,0.412671,-0.728809,-0.994397,-0.892543,-0.132296,-1.093925,-0.695517,...,0.137165,3.971853,-0.980071,-0.602848,-1.391463,1.763449,-0.070073,1.569996,-1.101390,-0.732631
206525,1.821379,1.235757,0.221342,1.378470,0.139262,2.154329,0.479456,0.281038,-0.560107,1.577744,...,2.151850,1.750859,1.175570,-0.649815,1.331331,1.466642,1.690066,1.386581,2.638980,0.660251
146533,1.656025,0.933225,1.614382,-0.344818,0.310732,2.457255,1.002415,0.519500,-0.491960,0.054950,...,1.305682,0.006832,0.956788,-0.161354,2.071697,-0.044374,-0.431244,-0.846505,1.292596,-1.024687
194746,-0.918764,0.273605,-0.657688,-0.067072,-0.621640,-0.772536,-0.486481,0.137961,-0.491960,-0.476934,...,0.016284,0.451031,-0.143553,-0.969194,-0.388072,-1.069706,-0.536396,0.079744,-0.490755,-0.081122


# 2. Train a SVM Classifier

In [9]:
for feature, feature_val in zip(df_X_tr, df_X_val):
    name = feature['name']
    data = feature['data']
    data_val = feature_val['data']
    
    #CREATE AND TRAIN 
    svc = SVC(C = 0.001, kernel = 'rbf', gamma = 0.001, max_iter = 10000)
    svc.fit(data, df_y_train['class'].values)
    
    #SCORING
    print("Score for " + name + ": ", svc.score(data_val, df_y_val))
    
    #SAVING THE CLASSIFICATOR 
    joblib.dump(svc,'output/SVM RFE (non linear - all features)' + name + '.pkl')

Score for area:  0.6893491124260355
Score for meancurv:  0.5828402366863905
Score for thickness:  0.5887573964497042
Score for volume:  0.7041420118343196
